In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import networkx as nx
import datgan

from sklearn.preprocessing import OrdinalEncoder

from datgan import stats_assessment
from datgan import ml_assessment, transform_results

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [50]:
df_orig = pd.read_csv('../../data/LPMC/trips.csv', index_col=False)

continuous_columns = ['start_time_linear', 'age', 'distance', 'dur_walking', 'dur_cycling', 'dur_pt','dur_driving', 'driving_traffic_percent']

cond_inputs = ["age", "female", "hh_borough"]

synth_files = []

for i in range(5):
    synth_files.append('../../data/synthetic/test/DATGAN_{:02d}.csv'.format(i+1))
    synth_files.append('../../data/synthetic/test/DATGAN2_{:02d}.csv'.format(i+1))

    synth_files.append('../../data/synthetic/test/ciDATGAN_{:02d}.csv'.format(i+1))
    synth_files.append('../../data/synthetic/test/ciDATGAN2_{:02d}.csv'.format(i+1))
    synth_files.append('../../data/synthetic/test/ciDATGAN3_{:02d}.csv'.format(i+1))

len_df = len(df_orig)

In [51]:
results_path = './results/'

if not os.path.exists(results_path):
    os.makedirs(results_path)

In [52]:
stats_str = ['mae', 'rmse', 'r2', 'srmse', 'corr']

# First level

In [53]:
pickle_name = 'stats_first_level.pickle'
aggregation_level = 1

first_lvl_stats = {}

try:
    first_lvl_stats = pickle.load(open(results_path + pickle_name, 'rb'))
    print('Found previous pickle file, using that')
except:
    print('No previous results found, starting fresh')

Found previous pickle file, using that


In [54]:
for i, f in enumerate(synth_files):

    file_name = f.split('/')[-1].split('.')[0]

    if file_name in first_lvl_stats:
        print("Results for file \033[1m{}\033[0m ({}/{}) already exists!".format(file_name, i+1, len(synth_files)))
    else:
        print("Preparing stats for file \033[1m{}\033[0m ({}/{})".format(file_name, i+1, len(synth_files)))

        first_lvl_stats[file_name] = {}

        df_synth = pd.read_csv(f, index_col=False)
        df_synth = df_synth.sample(len_df).reset_index(drop=True)

        stats = stats_assessment(df_orig, df_synth, continuous_columns, aggregation_level, ignore_cols=cond_inputs)

        first_lvl_stats[file_name] = stats

    pickle.dump(first_lvl_stats, open(results_path + pickle_name, 'wb'))

print("\033[1mFINISHED!\033[0m")

Results for file DATGAN_01 (1/25) already exists!
Results for file DATGAN2_01 (2/25) already exists!
Results for file ciDATGAN_01 (3/25) already exists!
Results for file ciDATGAN2_01 (4/25) already exists!
Preparing stats for file ciDATGAN3_01 (5/25)
Results for file DATGAN_02 (6/25) already exists!
Results for file DATGAN2_02 (7/25) already exists!
Results for file ciDATGAN_02 (8/25) already exists!
Results for file ciDATGAN2_02 (9/25) already exists!
Preparing stats for file ciDATGAN3_02 (10/25)
Results for file DATGAN_03 (11/25) already exists!
Results for file DATGAN2_03 (12/25) already exists!
Results for file ciDATGAN_03 (13/25) already exists!
Results for file ciDATGAN2_03 (14/25) already exists!
Preparing stats for file ciDATGAN3_03 (15/25)
Results for file DATGAN_04 (16/25) already exists!
Results for file DATGAN2_04 (17/25) already exists!
Results for file ciDATGAN_04 (18/25) already exists!
Results for file ciDATGAN2_04 (19/25) already exists!
Preparing stats for file ciDATG

In [55]:
res = {}

for test in ['all', 'cont', 'cat']:

    res[test] = {}

    if test == 'all':
        cols = df_orig.columns
    elif test == 'cont':
        cols = continuous_columns
    elif test == 'cat':
        cols = set(df_orig.columns) - set(continuous_columns)

    cols = set(cols) - set(cond_inputs)

    for s in stats_str:
        res[test][s] = {}

    for m in first_lvl_stats.keys():
        for s in stats_str:
            res[test][s][m] = []

            for c in cols:
                res[test][s][m].append(first_lvl_stats[m][c][s])

In [56]:
avg = {}

for test in ['all', 'cont', 'cat']:

    avg[test] = {}

    for s in stats_str:
        avg[test][s] = {}

        for m in first_lvl_stats.keys():
            avg[test][s][m] = {
                'mean': np.mean(res[test][s][m]),
                'std': np.std(res[test][s][m])
            }

In [57]:
for test in ['all', 'cont', 'cat']:

    if test == 'all':
        str_ = 'on all columns'
    elif test == 'cont':
        str_ = 'on continuous columns'
    elif test == 'cat':
        str_ = 'on categorical columns'

    for s in ['srmse']:#stats:
        print('Ranking {} based on {}:'.format(str_, s.upper()))

        if s in ['r2', 'corr']:
            sorted_dct = {k: v for k, v in sorted(avg[test][s].items(), key=lambda item: item[1]['mean'])[::-1]}
        else:
            sorted_dct = {k: v for k, v in sorted(avg[test][s].items(), key=lambda item: item[1]['mean'])}

        for i, item in enumerate(sorted_dct):
            print('  {:>2}. {:<15} - {:.2e} ± {:.2e}'.format(i+1, item, sorted_dct[item]['mean'], sorted_dct[item]['std']))
        print()

Ranking on all columns based on SRMSE:
   1. DATGAN2_05      - 8.04e-02 ± 6.79e-02
   2. DATGAN2_02      - 8.36e-02 ± 7.23e-02
   3. DATGAN2_04      - 8.41e-02 ± 7.46e-02
   4. DATGAN2_03      - 8.43e-02 ± 7.50e-02
   5. DATGAN2_01      - 8.52e-02 ± 7.44e-02
   6. ciDATGAN_03     - 8.78e-02 ± 8.02e-02
   7. DATGAN_02       - 9.13e-02 ± 5.36e-02
   8. DATGAN_01       - 9.49e-02 ± 5.42e-02
   9. ciDATGAN_05     - 9.53e-02 ± 8.10e-02
  10. ciDATGAN3_04    - 9.54e-02 ± 8.29e-02
  11. ciDATGAN_01     - 9.65e-02 ± 8.09e-02
  12. ciDATGAN_04     - 9.69e-02 ± 8.03e-02
  13. ciDATGAN_02     - 9.72e-02 ± 8.12e-02
  14. DATGAN_04       - 9.83e-02 ± 5.50e-02
  15. ciDATGAN3_03    - 9.83e-02 ± 8.74e-02
  16. ciDATGAN3_01    - 9.83e-02 ± 8.93e-02
  17. DATGAN_05       - 9.93e-02 ± 5.73e-02
  18. DATGAN_03       - 9.93e-02 ± 5.83e-02
  19. ciDATGAN3_05    - 9.96e-02 ± 8.60e-02
  20. ciDATGAN3_02    - 1.03e-01 ± 9.15e-02
  21. ciDATGAN2_02    - 1.17e-01 ± 1.17e-01
  22. ciDATGAN2_03    - 1.17e-01 ± 1.

# Second level

In [58]:
pickle_name = 'stats_second_level.pickle'
aggregation_level = 2

second_lvl_stats = {}

try:
    second_lvl_stats = pickle.load(open(results_path + pickle_name, 'rb'))
    print('Found previous pickel file, using that')
except:
    print('No previous results found, starting fresh')

Found previous pickel file, using that


In [59]:
for i, f in enumerate(synth_files):

    file_name = f.split('/')[-1].split('.')[0]

    if file_name in second_lvl_stats:
        print("Results for file \033[1m{}\033[0m ({}/{}) already exists!".format(file_name, i+1, len(synth_files)))
    else:
        print("Preparing stats for file \033[1m{}\033[0m ({}/{})".format(file_name, i+1, len(synth_files)))

        second_lvl_stats[file_name] = {}

        df_synth = pd.read_csv(f, index_col=False)
        df_synth = df_synth.sample(len_df).reset_index(drop=True)

        stats = stats_assessment(df_orig, df_synth, continuous_columns, aggregation_level, ignore_cols=cond_inputs)

        second_lvl_stats[file_name] = stats

    pickle.dump(second_lvl_stats, open(results_path + pickle_name, 'wb'))

print("\033[1mFINISHED!\033[0m")

Results for file DATGAN_01 (1/25) already exists!
Results for file DATGAN2_01 (2/25) already exists!
Results for file ciDATGAN_01 (3/25) already exists!
Results for file ciDATGAN2_01 (4/25) already exists!
Preparing stats for file ciDATGAN3_01 (5/25)
Results for file DATGAN_02 (6/25) already exists!
Results for file DATGAN2_02 (7/25) already exists!
Results for file ciDATGAN_02 (8/25) already exists!
Results for file ciDATGAN2_02 (9/25) already exists!
Preparing stats for file ciDATGAN3_02 (10/25)
Results for file DATGAN_03 (11/25) already exists!
Results for file DATGAN2_03 (12/25) already exists!
Results for file ciDATGAN_03 (13/25) already exists!
Results for file ciDATGAN2_03 (14/25) already exists!
Preparing stats for file ciDATGAN3_03 (15/25)
Results for file DATGAN_04 (16/25) already exists!
Results for file DATGAN2_04 (17/25) already exists!
Results for file ciDATGAN_04 (18/25) already exists!
Results for file ciDATGAN2_04 (19/25) already exists!
Preparing stats for file ciDATG

In [60]:
res = {}

for s in stats_str:
    res[s] = {}

for m in second_lvl_stats.keys():

    for s in stats_str:
        res[s][m] = []

        for c in second_lvl_stats[m].keys():
            res[s][m].append(second_lvl_stats[m][c][s])

In [61]:
avg = {}

for s in stats_str:
    avg[s] = {}

    for m in first_lvl_stats.keys():
        avg[s][m] = {
            'mean': np.mean(res[s][m]),
            'std': np.std(res[s][m])
        }

In [62]:
for s in ['srmse']:#stats:
    print('Ranking based on {} for aggregation level {}:'.format(s.upper(), aggregation_level))

    if s in ['r2', 'corr']:
        sorted_dct = {k: v for k, v in sorted(avg[s].items(), key=lambda item: item[1]['mean'])[::-1]}
    else:
        sorted_dct = {k: v for k, v in sorted(avg[s].items(), key=lambda item: item[1]['mean'])}

    for i, item in enumerate(sorted_dct):
        print('  {:>2}. {:<15} - {:.2e} ± {:.2e}'.format(i+1, item, sorted_dct[item]['mean'], sorted_dct[item]['std']))
    print()

Ranking based on SRMSE for aggregation level 2:
   1. DATGAN_02       - 2.63e-01 ± 1.18e-01
   2. ciDATGAN_03     - 2.64e-01 ± 1.76e-01
   3. DATGAN_01       - 2.66e-01 ± 1.20e-01
   4. ciDATGAN_01     - 2.69e-01 ± 1.66e-01
   5. DATGAN2_05      - 2.69e-01 ± 1.56e-01
   6. DATGAN2_03      - 2.71e-01 ± 1.54e-01
   7. ciDATGAN_05     - 2.71e-01 ± 1.72e-01
   8. DATGAN_04       - 2.72e-01 ± 1.17e-01
   9. ciDATGAN_04     - 2.74e-01 ± 1.68e-01
  10. ciDATGAN_02     - 2.74e-01 ± 1.71e-01
  11. DATGAN_03       - 2.74e-01 ± 1.22e-01
  12. DATGAN2_01      - 2.74e-01 ± 1.61e-01
  13. DATGAN2_02      - 2.76e-01 ± 1.57e-01
  14. DATGAN_05       - 2.76e-01 ± 1.21e-01
  15. DATGAN2_04      - 2.83e-01 ± 1.70e-01
  16. ciDATGAN3_04    - 2.83e-01 ± 1.80e-01
  17. ciDATGAN3_01    - 2.86e-01 ± 1.82e-01
  18. ciDATGAN3_05    - 2.90e-01 ± 1.84e-01
  19. ciDATGAN3_03    - 2.91e-01 ± 1.86e-01
  20. ciDATGAN3_02    - 2.99e-01 ± 1.92e-01
  21. ciDATGAN2_02    - 3.14e-01 ± 2.18e-01
  22. ciDATGAN2_03    - 3.15

# ML efficacy

In [63]:
def check_low_appearing_vars(df):

    for c in df.columns:
        val = df[c].value_counts()
        if len(val) < 20:
            val = val/len(df)
            if any(val < 0.01) and c != 'choice':
                print('Variable {}: '.format(c))
                for idx, v in zip(val.index, val):
                    if v < 0.01:
                        print('  {} - {:.2f}% ({:d})'.format(idx, 100*v, int(v*len(df))))
                print()

def replace_low_appearing_values(df):

    dct_ = {}
    for i in df['hh_vehicles'].unique():
        if i >= 3:
            dct_[i] = '3+'
        else:
            dct_[i] = str(i)
    df['hh_vehicles'].replace(dct_, inplace=True)

    dct_ = {}
    for i in df['hh_people'].unique():
        if i >= 6:
            dct_[i] = '6+'
        else:
            dct_[i] = str(i)
    df['hh_people'].replace(dct_, inplace=True)

In [64]:
check_low_appearing_vars(df_orig)

Variable hh_vehicles: 
  4 - 0.52% (88)
  5 - 0.12% (21)
  6 - 0.02% (3)
  8 - 0.02% (3)
  7 - 0.01% (2)

Variable hh_people: 
  7 - 0.70% (118)
  8 - 0.18% (31)
  9 - 0.08% (13)
  10 - 0.03% (5)
  11 - 0.02% (3)



In [65]:
replace_low_appearing_values(df_orig)

In [66]:
check_low_appearing_vars(df_orig)

In [67]:
categorical_columns = list(set(df_orig.columns) - set(continuous_columns))

In [68]:
enc = OrdinalEncoder()
df_orig[categorical_columns] = enc.fit_transform(df_orig[categorical_columns])

In [69]:
pickle_name = 'ml_efficacy.pickle'

cv_modelscores = {}

try:
    cv_modelscores = pickle.load(open(results_path + pickle_name, 'rb'))
    print('Found previous pickel file, using that')
except:
    print('No previous results found, starting fresh')

Found previous pickel file, using that


In [70]:
for i, f in enumerate(synth_files):

    file_name = f.split('/')[-1].split('.')[0]

    if file_name in cv_modelscores:
        print("Results for file \033[1m{}\033[0m ({}/{}) already exists!".format(file_name, i+1, len(synth_files)))
    else:
        print("Preparing stats for file \033[1m{}\033[0m ({}/{})".format(file_name, i+1, len(synth_files)))

        cv_modelscores[file_name] = {}

        # Load the synthetic dataset
        df_synth = pd.read_csv(f, index_col=False)
        df_synth = df_synth.sample(len_df).reset_index(drop=True)

        # Replace the values rarely appearing
        replace_low_appearing_values(df_synth)

        # Encode the synthetic dataset
        df_synth[categorical_columns] = enc.transform(df_synth[categorical_columns])

        res = ml_assessment(df_orig, df_synth, continuous_columns, categorical_columns, ignore_cols=cond_inputs)

        cv_modelscores[file_name] = res

    pickle.dump(cv_modelscores, open(results_path + pickle_name, 'wb'))

print("\033[1mFINISHED!\033[0m")

Results for file DATGAN_01 (1/25) already exists!
Results for file DATGAN2_01 (2/25) already exists!
Results for file ciDATGAN_01 (3/25) already exists!
Results for file ciDATGAN2_01 (4/25) already exists!
Preparing stats for file ciDATGAN3_01 (5/25)
Results for file DATGAN_02 (6/25) already exists!
Results for file DATGAN2_02 (7/25) already exists!
Results for file ciDATGAN_02 (8/25) already exists!
Results for file ciDATGAN2_02 (9/25) already exists!
Preparing stats for file ciDATGAN3_02 (10/25)
Results for file DATGAN_03 (11/25) already exists!
Results for file DATGAN2_03 (12/25) already exists!
Results for file ciDATGAN_03 (13/25) already exists!
Results for file ciDATGAN2_03 (14/25) already exists!
Preparing stats for file ciDATGAN3_03 (15/25)
Results for file DATGAN_04 (16/25) already exists!
Results for file DATGAN2_04 (17/25) already exists!
Results for file ciDATGAN_04 (18/25) already exists!
Results for file ciDATGAN2_04 (19/25) already exists!
Preparing stats for file ciDATG

In [71]:
if 'original' in cv_modelscores:
    print("Results for file \033[1m{}\033[0m already exists!".format('original'))
else:
    print("Preparing stats for file \033[1m{}\033[0m".format('original'))

    res = ml_assessment(df_orig, df_orig, continuous_columns, categorical_columns, ignore_cols=cond_inputs)
    cv_modelscores['original'] = res
    pickle.dump(cv_modelscores, open(results_path + pickle_name, 'wb'))
    print("\033[1mFINISHED!\033[0m")

Results for file original already exists!


In [72]:
cont_sorted, cat_sorted = transform_results(cv_modelscores, continuous_columns, categorical_columns, ignore_cols=cond_inputs)

In [73]:
i=1
print('   | {:<30} | {:<30}'.format('categorical', 'continuous'))
print('-----------------------------------------------------------')
for a, b in zip(cat_sorted, cont_sorted):
    print('{:>2} | {:<30} | {:<30}'.format(i, '{:<12}: {:.3f}'.format(a[0], a[1]), '{:<12}: {:.3f}'.format(b[0], b[1])))
    i+=1

   | categorical                    | continuous                    
-----------------------------------------------------------
 1 | original    : -1.274           | original    : 5.381           
 2 | ciDATGAN2_01: 0.603            | DATGAN_03   : 13.130          
 3 | ciDATGAN3_05: 0.611            | DATGAN_04   : 13.208          
 4 | ciDATGAN3_02: 0.615            | DATGAN2_01  : 13.220          
 5 | ciDATGAN3_03: 0.621            | DATGAN_02   : 13.284          
 6 | ciDATGAN2_03: 0.623            | DATGAN2_03  : 13.332          
 7 | ciDATGAN2_04: 0.623            | DATGAN2_05  : 13.454          
 8 | ciDATGAN3_01: 0.624            | DATGAN_01   : 13.468          
 9 | ciDATGAN3_04: 0.631            | DATGAN_05   : 13.546          
10 | ciDATGAN2_02: 0.633            | DATGAN2_02  : 13.796          
11 | ciDATGAN2_05: 0.635            | DATGAN2_04  : 13.803          
12 | ciDATGAN_05 : 0.650            | ciDATGAN_05 : 17.163          
13 | ciDATGAN_04 : 0.650            | ciDAT